In [1]:
import cenpy
import contextily as ctx 
import segregation
%matplotlib inline
import matplotlib as mpl

In [2]:
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
from contextily import add_basemap
import folium 

In [3]:
import geopandas as gpd 
from census import Census 
from us import states
from shapely.geometry import Polygon

In [4]:
import pandas as pd

**Census Api**

In [5]:
acs=cenpy.products.ACS(year="latest")  # type: ignore
acs.filter_tables('RACE', by='description')


,description,columns
table_name,,
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
B02009,BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINAT...,[B02009_001E]
B02010,AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN ...,[B02010_001E]
B02011,ASIAN ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02011_001E]
B02012,NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALO...,[B02012_001E]
B02013,SOME OTHER RACE ALONE OR IN COMBINATION WITH O...,[B02013_001E]
B03002,HISPANIC OR LATINO ORIGIN BY RACE,"[B03002_001E, B03002_002E, B03002_003E, B03002..."
B25006,RACE OF HOUSEHOLDER,"[B25006_001E, B25006_002E, B25006_003E, B25006..."


In [6]:
c = Census('f5e7cf46eaa83ae8323e6051f74d9ab363d0b957')

In [7]:
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
bx_census_tract = c.acs5.state_county_tract(fields = ('NAME', 'B02001_003E','B02001_001E' ),
                                      state_fips = states.TX.fips, 
                                      county_fips = "029",
                                      tract = "*",
                                      year = 2019)

In [8]:
bx_df = pd.DataFrame(bx_census_tract)

In [9]:
print(bx_df.head(2))
print('Shape: ', bx_df.shape)

                                     NAME  B02001_003E  B02001_001E state  \
0  Census Tract 1109, Bexar County, Texas         46.0       1669.0    48   
1  Census Tract 1110, Bexar County, Texas        171.0       2948.0    48   

  county   tract  
0    029  110900  
1    029  111000  
Shape:  (366, 6)


access shapefile of bexar county

In [10]:
usa=gpd.read_file('../tl_2019_us_county/tl_2019_us_county.shp')
print(usa)

     STATEFP COUNTYFP  COUNTYNS  GEOID       NAME          NAMELSAD LSAD  \
0         31      039  00835841  31039     Cuming     Cuming County   06   
1         53      069  01513275  53069  Wahkiakum  Wahkiakum County   06   
2         35      011  00933054  35011    De Baca    De Baca County   06   
3         31      109  00835876  31109  Lancaster  Lancaster County   06   
4         31      129  00835886  31129   Nuckolls   Nuckolls County   06   
...      ...      ...       ...    ...        ...               ...  ...   
3228      13      123  00351260  13123     Gilmer     Gilmer County   06   
3229      27      135  00659513  27135     Roseau     Roseau County   06   
3230      28      089  00695768  28089    Madison    Madison County   06   
3231      48      227  01383899  48227     Howard     Howard County   06   
3232      54      099  01550056  54099      Wayne      Wayne County   06   

     CLASSFP  MTFCC CSAFP CBSAFP METDIVFP FUNCSTAT       ALAND    AWATER  \
0         H

In [11]:
usa.head(2)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."


In [12]:
bexar= usa()

NameError: name 'coliu' is not defined

In [ ]:
texas_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_48_tract.zip")
print(texas_tract)

In [ ]:
texas_tract = texas_tract.to_crs(epsg=32617)

In [ ]:
print(texas_tract.head(2))

print('Shape: ', texas_tract.shape)

# Check shapefile projection
print("\nThe shapefile projection is: {}".format(texas_tract.crs))

In [ ]:
texas_tract.plot('COUNTYFP', legend=True, figsize=(20,20))

performing dataframe operations

In [ ]:
# Combine state, county, and tract columns together to create a new string and assign to new column
bx_df["GEOID"] = bx_df["state"] + bx_df["county"] + bx_df["tract"]

In [ ]:
bx_df.head(100)

**Bexar County RACE Population (percentage)**

In [ ]:
race = ['B02001_001E', #RACE all
                'B02001_003E' #BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINAT...	
                ]
race

In [ ]:
bx_race_black = acs.from_county('Bexar County, TX', variables=race)
bx_race_black

In [ ]:
bx_race_black_basemap, bx_race_black_extent = ctx.bounds2img(*bx_race_black.total_bounds, zoom=10,   
                                            source=ctx.providers.OpenStreetMap.Mapnik )

In [ ]:
f,ax = plt.subplots(1,1, figsize=(20,16))

plt.rcParams['legend.title_fontsize'] = 'medium'

plt.rcParams['font.size'] = 16

plt.rcParams['font.weight'] = 600

ax.imshow(bx_race_black_basemap, extent=bx_race_black_extent, interpolation='sinc')
bx_race_black['pct_black'] = bx_race_black.eval('B02001_003E / B02001_001E')*100  # type: ignore

bx_race_black.plot('pct_black', edgecolor='white',linewidth=.2, cmap='CMRmap_r', ax = ax,  # type: ignore
                    alpha=.7, legend=True,
                    legend_kwds={ 'label': "Percentage of Black/African American residences living in Bexar County",'orientation': "horizontal", 'pad':.01})  # type: ignore

plt.title(
    "\nBexar County\n Black and African American Population (percentage)\n",
    fontsize=20,
    loc='center',
    fontweight='bold',
    #style='italic',
    family='monospace')

plt.style.use('dark_background')

ax.tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)